In [3]:
from skillcorner.client import SkillcornerClient

import pandas as pd

import numpy as np

import os

secret_password = os.getenv("mdp_skillcorner")
client = SkillcornerClient(username = "Nathan.talbot@etu.uca.fr", password = secret_password)

In [2]:
import pandas as pd

liste_dico = [{"match_drop" : [1550555, 1546206],
                "ranking" : ["AJ Auxerre", "Angers SCO", "AS Saint-Étienne", "Rodez Aveyron", "Paris FC", "SM Caen", "Stade Lavallois Mayenne FC",
           "Amiens Sporting Club", "En Avant de Guingamp", "Pau FC", "Grenoble Foot 38", "Girondins de Bordeaux", "SC Bastia",
           "FC Annecy", "AC Ajaccio", "Dunkerque", "ES Troyes AC", "US Quevilly-Rouen", "US Concarneau", "Valenciennes FC"],
           "annee" : "2023_2024"},
           {"match_drop" : [],
                "ranking" : ["Le Havre AC", "FC Metz", "Girondins de Bordeaux", "SC Bastia", "SM Caen", "En Avant de Guingamp", "Paris FC",
           "AS Saint-Étienne", "FC Sochaux-Montbéliard", "Grenoble Foot 38", "US Quevilly-Rouen", "Amiens Sporting Club", "Pau FC",
           "Rodez Aveyron", "Stade Lavallois Mayenne FC", "Valenciennes FC", "FC Annecy", "Dijon FCO", "Nîmes Olympique", "Chamois Niortais FC"],
           "annee" : "2022_2023"},
           {"match_drop" : [129364, 128946],
                "ranking" : ["Toulouse FC", "AC Ajaccio", "AJ Auxerre", "Paris FC", "FC Sochaux-Montbéliard", "En Avant de Guingamp",
                             "SM Caen", "Le Havre AC", "Nîmes Olympique", "Pau FC", "Dijon FCO", "SC Bastia", "Chamois Niortais FC", 
                             "Amiens Sporting Club", "Grenoble Foot 38", "Valenciennes FC", "Rodez Aveyron", "US Quevilly-Rouen",
                             "Dunkerque", "AS Nancy-Lorraine"],
           "annee" : "2021_2022"}
           ]

dico = liste_dico[0]


In [5]:
data_import = pd.read_excel(f"../../../../Data_file/Métriques Team sur une Saison Ligue 2 SB + SK/{dico["annee"]}/Skill Corner/data_passes.xlsx",
                            index_col = 0)

In [16]:
data = data_import[~(data_import.match_id.isin(dico["match_drop"]))]
data = data[data.quality_check == True]
data.fillna(0, inplace = True)

drop = ["quality_check", "player_id", "player_name", "short_name", "player_birthdate", "match_name", "match_date", "team_id",
    "competition_id", "competition_name", "season_id", "season_name", "competition_edition_id", "position", "group", "result",
    "venue", "third", "channel", "adjusted_min_tip_per_match"]
data.drop(drop, inplace = True, axis = 1)
sample = data.columns[["sample" in i for i in data.columns]]
data.drop(sample, inplace = True, axis = 1)

# AJOUT NUMÉRO DE JOURNÉE AU DATAFRAME
match_round = pd.read_excel(f"../../../../Data_file/Métriques Team sur une Saison Ligue 2 SB + SK/{dico["annee"]}/Skill Corner/match_round.xlsx",
                            index_col = 0)
match_round.drop(index = dico["match_drop"], inplace = True)
data = data.merge(match_round, right_on = "id", left_on = "match_id").drop("match_id", axis = 1)

# modif métriques ratios

col_ratio_rep = data.columns[[("ratio" in i) and ("per_100_pass_opportunities" in i or "per_match" in i) for i in data.columns]]
data.drop(col_ratio_rep, inplace = True, axis = 1)

met_ratio = data.columns[["ratio" in i for i in data.columns]]
data = data.groupby(["Journée", "team_name"], as_index = True)
nb_joueur_match = data.apply(len)
data = data.sum()
data[met_ratio] = data[met_ratio].divide(nb_joueur_match, axis = 0)

nb_minute_match = data.pop("minutes_played_per_match")
data = data.multiply(900/nb_minute_match, axis = 0).reindex(dico["ranking"], axis = 0, level = 1)

C:\Users\fabie\AppData\Local\Temp\ipykernel_11872\3607474491.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  nb_joueur_match = data.apply(len)


In [40]:
idx = pd.IndexSlice
data.loc[:, data.index.levels[1][:2], :]

count_opportunities_to_pass_to_runs_in_behind_per_match  \
Journée team_name                                                             
1       AJ Auxerre                                          34.177443         
        Angers SCO                                          39.746526         
2       AJ Auxerre                                          30.716724         
        Angers SCO                                          21.100917         
3       AJ Auxerre                                          21.992499         
...                                                               ...         
36      Angers SCO                                          18.329939         
37      AJ Auxerre                                          35.359801         
        Angers SCO                                          16.368762         
38      AJ Auxerre                                          46.766169         
        Angers SCO                                          25.506991         

                    count_pass_attempts_to_runs_in_behind_per_match  \
Journée team_name                                                     
1       AJ Auxerre                                        12.591689   
        Angers SCO                                        25.881459   
2       AJ Auxerre                                         9.937764   
        Angers SCO                                         9.174312   
3       AJ Auxerre                                         9.561956   
...                                                             ...   
36      Angers SCO                                        11.914460   
37      AJ Auxerre                                         8.374690   
        Angers SCO                                        10.003132   
38      AJ Auxerre                                        22.885572   
        Angers SCO                                        18.894067   

                    pass_opportunities_to_runs_in_behind_threat_per_match  \
Journée team_name                                                           
1       AJ Auxerre                                           1.250175       
        Angers SCO                                           0.785687       
2       AJ Auxerre                                           0.849227       
        Angers SCO                                           0.669725       
3       AJ Auxerre                                           0.640651       
...                                                               ...       
36      Angers SCO                                           0.898167       
37      AJ Auxerre                                           1.023573       
        Angers SCO                                           0.500157       
38      AJ Auxerre                                           1.661692       
        Angers SCO                                           0.765210       

                    runs_in_behind_to_which_pass_attempted_threat_per_match  \
Journée team_name                                                             
1       AJ Auxerre                                           0.530650         
        Angers SCO                                           0.536116         
2       AJ Auxerre                                           0.316202         
        Angers SCO                                           0.201835         
3       AJ Auxerre                                           0.229487         
...                                                               ...         
36      Angers SCO                                           0.577393         
37      AJ Auxerre                                           0.186104         
        Angers SCO                                           0.300094         
38      AJ Auxerre                                           0.805970         
        Angers SCO                                           0.425117         

                    count_runs_in_behi

In [84]:
import pandas as pd

liste_dico = [{"match_drop" : [1550555, 1546206],
                "ranking" : ["AJ Auxerre", "Angers SCO", "AS Saint-Étienne", "Rodez Aveyron", "Paris FC", "SM Caen", "Stade Lavallois Mayenne FC",
           "Amiens Sporting Club", "En Avant de Guingamp", "Pau FC", "Grenoble Foot 38", "Girondins de Bordeaux", "SC Bastia",
           "FC Annecy", "AC Ajaccio", "Dunkerque", "ES Troyes AC", "US Quevilly-Rouen", "US Concarneau", "Valenciennes FC"],
           "annee" : "2023_2024"},
           {"match_drop" : [],
                "ranking" : ["Le Havre AC", "FC Metz", "Girondins de Bordeaux", "SC Bastia", "SM Caen", "En Avant de Guingamp", "Paris FC",
           "AS Saint-Étienne", "FC Sochaux-Montbéliard", "Grenoble Foot 38", "US Quevilly-Rouen", "Amiens Sporting Club", "Pau FC",
           "Rodez Aveyron", "Stade Lavallois Mayenne FC", "Valenciennes FC", "FC Annecy", "Dijon FCO", "Nîmes Olympique", "Chamois Niortais FC"],
           "annee" : "2022_2023"},
           {"match_drop" : [129364, 128946],
                "ranking" : ["Toulouse FC", "AC Ajaccio", "AJ Auxerre", "Paris FC", "FC Sochaux-Montbéliard", "En Avant de Guingamp",
                             "SM Caen", "Le Havre AC", "Nîmes Olympique", "Pau FC", "Dijon FCO", "SC Bastia", "Chamois Niortais FC", 
                             "Amiens Sporting Club", "Grenoble Foot 38", "Valenciennes FC", "Rodez Aveyron", "US Quevilly-Rouen",
                             "Dunkerque", "AS Nancy-Lorraine"],
           "annee" : "2021_2022"}
           ]

for dico in liste_dico :
     data_import = pd.read_excel(f"../../../../Data_file/Métriques Team sur une Saison Ligue 2 SB + SK/{dico["annee"]}/Skill Corner/data_passes.xlsx", index_col = 0)
     data = data_import[~(data_import.match_id.isin(dico["match_drop"]))]
     data = data[data.quality_check == True]
     data.fillna(0, inplace = True)

     drop = ["quality_check", "player_id", "player_name", "short_name", "player_birthdate", "match_name", "match_date", "team_id",
          "competition_id", "competition_name", "season_id", "season_name", "competition_edition_id", "position", "group", "result", "venue",
          "third", "channel", "adjusted_min_tip_per_match"]
     data.drop(drop, inplace = True, axis = 1)
     sample = data.columns[["sample" in i for i in data.columns]]
     data.drop(sample, inplace = True, axis = 1)

     # AJOUT NUMÉRO DE JOURNÉE AU DATAFRAME
     match_round = pd.read_excel(f"../../../../Data_file/Métriques Team sur une Saison Ligue 2 SB + SK/{dico["annee"]}/Skill Corner/match_round.xlsx",
                                   index_col = 0)
     match_round.drop(index = dico["match_drop"], inplace = True)
     data = data.merge(match_round, right_on = "id", left_on = "match_id").drop("match_id", axis = 1)

     # modif métriques ratios
     met_ratio = data.columns[["ratio" in i for i in data.columns]]
     data = data.groupby(["Journée", "team_name"], as_index = True)
     nb_joueur_match = data.apply(len)
     data = data.sum()
     data[met_ratio] = data[met_ratio].divide(nb_joueur_match, axis = 0)

     nb_minute_match = data.pop("minutes_played_per_match")
     data = data.multiply(900/nb_minute_match, axis = 0)

     moyenne_top20 = data.groupby("Journée").mean()
     moyenne_top5 = data[data.index.get_level_values("team_name").isin(dico["ranking"])].groupby("Journée").mean()
     moyenne_bottom15 = data[~data.index.get_level_values("team_name").isin(dico["ranking"])].groupby("Journée").mean()
     break

C:\Users\fabie\AppData\Local\Temp\ipykernel_16768\511126347.py:43: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  nb_joueur_match = data.apply(len)


In [92]:
data[~(data.index.get_level_values("team_name").isin(dico["ranking"]))]

,,count_opportunities_to_pass_to_runs_in_behind_per_match,count_pass_attempts_to_runs_in_behind_per_match,pass_opportunities_to_runs_in_behind_threat_per_match,runs_in_behind_to_which_pass_attempted_threat_per_match,pass_completion_ratio_to_runs_in_behind,count_runs_in_behind_by_teammate_per_match,runs_in_behind_to_which_pass_completed_threat_per_match,count_completed_pass_to_runs_in_behind_per_match,count_completed_pass_to_runs_in_behind_leading_to_shot_per_match,count_completed_pass_to_runs_in_behind_leading_to_goal_per_match,...,cross_receiver_runs_to_which_pass_attempted_threat_per_match,pass_completion_ratio_to_cross_receiver_runs,count_cross_receiver_runs_by_teammate_per_match,cross_receiver_runs_to_which_pass_completed_threat_per_match,count_completed_pass_to_cross_receiver_runs_per_match,count_completed_pass_to_cross_receiver_runs_leading_to_shot_per_match,count_completed_pass_to_cross_receiver_runs_leading_to_goal_per_match,count_pass_opportunities_to_dangerous_cross_receiver_runs_per_match,count_pass_attempts_to_dangerous_cross_receiver_runs_per_match,count_completed_pass_to_dangerous_cross_receiver_runs_per_match
Journée,team_name,,,,,,,,,,,,,,,,,,,,,
